In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
#install hyperopt, ! as CLI command
!pip install hyperas

In [3]:
!pip install hyperopt==0.2.5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 966.0/966.0 KB 11.2 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: hyperopt
    Found existing installation: hyperopt 0.2.7
    Uninstalling hyperopt-0.2.7:
      Successfully uninstalled hyperopt-0.2.7


In [4]:
from hyperopt import hp
from hyperopt import Trials, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

def prepare_data():
    """
    prepare data
    """
    import numpy as np
    import pandas as pd
    from sklearn.model_selection import KFold
    from tensorflow.keras.utils import to_categorical
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Activation, Dropout
    
    #import data from train.csv into pd DataFrame
    train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    train_x = train.drop(['label'], axis = 1) #from train take outimage data
    train_y = train["label"]
    
    #separate train into training and validating
    kf = KFold(n_splits = 4,
              shuffle = True,
              random_state = 123)
    tr_idx, va_idx = list(kf.split(train_x))[0]
    tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
    
    #divided by 255 and trun into np.array
    tr_x = np.array(tr_x/255.0) 
    va_x = np.array(va_x/255.0)
    
    #one hot encoding
    tr_y = to_categorical(tr_y,10)
    va_y = to_categorical(va_y,10)
    
    return tr_x, tr_y, va_x, va_y


def create_model(tr_x, tr_y):
    """
    establish model
    """
    
    #build Sequential object
    model = Sequential()
    
    #setup 1st layer
    model.add(Dense({{choice([500,784])}},input_dim=tr_x.shape[1],activation = 'relu'))
    model.add(Dropout(0.4))
    
    #from 0,1,2 find the optimal number of layers
    if {{choice(['none', 'one', 'two'])}} == 'none':
        pass
    elif {{choice(['none', 'one', 'two'])}} == 'one':
        model.add(Dense({{choice([100,200])}},activation = 'relu'))
    elif {{choice(['none', 'one', 'two'])}} == 'two':
        model.add(Dense({{choice([100,200])}},activation = 'relu'))
        model.add(Dense({{choice([25,50])}},activation = 'relu'))
    
    #output layer
    model.add(Dense(10,activation = 'softmax'))
    
    #establish compiler
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = {{choice(['adam', 'rmsprop'])}},
                  metrics = ['accuracy'])
    
    #training epoch and batch size
    epoch = 10
    batch_size = 100
    
    #train
    result = model.fit(tr_x,
                       tr_y,
                       epochs = epoch,
                       batch_size = batch_size,
                       validation_data = (va_x, va_y),
                       verbose = 0)
    
    #find accuracy in searching
    validation_acc = np.amax(result.history['val_accuracy'])
    print('Accuracy in search: ', validation_acc)
    
    return {'loss': -validation_acc,
            'status': STATUS_OK,
            'model': model}

#execute search
nb_name = '__notebook_source__'
best_run, best_model = optim.minimize(model=create_model,
                                     data = prepare_data,
                                     algo = tpe.suggest,
                                     max_evals = 100,
                                     eval_space = True,
                                     notebook_name = nb_name,
                                     trials = Trials())

>>> Imports:
#coding=utf-8

>>> Hyperas search space:

def get_space():
    return {
        'Dense': hp.choice('Dense', [500,784]),
        'Dense_1': hp.choice('Dense_1', ['none', 'one', 'two']),
        'Dense_2': hp.choice('Dense_2', ['none', 'one', 'two']),
        'Dense_3': hp.choice('Dense_3', [100,200]),
        'Dense_4': hp.choice('Dense_4', ['none', 'one', 'two']),
        'Dense_5': hp.choice('Dense_5', [100,200]),
        'Dense_6': hp.choice('Dense_6', [25,50]),
        'optimizer': hp.choice('optimizer', ['adam', 'rmsprop']),
    }

>>> Data
  1: 
  2: """
  3: prepare data
  4: """
  5: import numpy as np
  6: import pandas as pd
  7: from sklearn.model_selection import KFold
  8: from tensorflow.keras.utils import to_categorical
  9: from tensorflow.keras.models import Sequential
 10: from tensorflow.keras.layers import Dense, Activation, Dropout
 11: 
 12: #import data from train.csv into pd DataFrame
 13: train = pd.read_csv('/kaggle/input/digit-recognizer/train.csv

2022-05-11 16:02:07.778255: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-11 16:02:07.880393: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-11 16:02:07.881179: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-05-11 16:02:07.882302: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Accuracy in search:                                    
0.9773333072662354                                     
Accuracy in search:                                                               
0.977904736995697                                                                 
Accuracy in search:                                                               
0.9785714149475098                                                               
Accuracy in search:                                                               
0.9778095483779907                                                                
Accuracy in search:                                                               
0.9795238375663757                                                                
Accuracy in search:                                                               
0.9780952334403992                                                                
Accuracy in search:                                        

In [6]:
print(best_model.summary())
print(best_run)

_, _, va_x, va_y = prepare_data()
val_loss, val_acc = best_model.evaluate(va_x, va_y)
print("val_loss:" , val_loss)
print("val_acc: ", val_acc)

Model: "sequential_95"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_271 (Dense)            (None, 784)               615440    
_________________________________________________________________
dropout_95 (Dropout)         (None, 784)               0         
_________________________________________________________________
dense_272 (Dense)            (None, 200)               157000    
_________________________________________________________________
dense_273 (Dense)            (None, 50)                10050     
_________________________________________________________________
dense_274 (Dense)            (None, 10)                510       
Total params: 783,000
Trainable params: 783,000
Non-trainable params: 0
_________________________________________________________________
None
{'Dense': 784, 'Dense_1': 'one', 'Dense_2': 'none', 'Dense_3': 200, 'Dense_4': 'two', 'Dense_5': 200, 'Dense